In [3]:
# -*- coding: utf-8 -*-
"""
Created on Fri Sep 14 23:22:19 2018

@author: Administrator
"""
import re
import json
import time
import random
import requests
from pyquery import PyQuery as pq
import hashlib

url_base = 'https://www.instagram.com/boopmynose/'
uri_old     = 'https://www.instagram.com/graphql/query/?query_hash=a5164aed103f24b03e7b7747a2d94e3c&variables=%7B%22id%22%3A%22{user_id}%22%2C%22first%22%3A12%2C%22after%22%3A%22{cursor}%22%7D'
uri_new = 'https://www.instagram.com/graphql/query/?query_hash=66eb9403e44cc12e5b5ecda48b667d41&variables=%7B%22id%22%3A%22{user_id}%22%2C%22first%22%3A12%2C%22after%22%3A%22{cursor}%22%7D'
uri = uri_new
headers = {
'Connection':'keep-alive',
'Host':'www.instagram.com',
'Referer':'https://www.instagram.com/instagram/',
'User-Agent':'Mozilla/5.0 (Windows NT 10.0; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/69.0.3497.100 Safari/537.36',
'X-Requested-With':'XMLHttpRequest'
}

proxy = {
    'http': 'http://127.0.0.1:1080',
    'https': 'http://127.0.0.1:1080'
}


def hashStr(strInfo):
    h = hashlib.md5()
    h.update(strInfo.encode("utf-8"))
    return h.hexdigest()

def get_html(url):
    try:
        response = requests.get(url, headers=headers, proxies=proxy)
        if response.status_code == 200:
            return response.text
        else:
            print('请求网页源代码错误, 错误状态码：', response.status_code)
    except Exception as e:
        print(e)
        return None


def get_json(headers,url):
    try:
        response = requests.get(url, headers=headers,proxies=proxy, timeout=10)
        if response.status_code == 200:
            return response.json()
        else:
            print('请求网页json错误, 错误状态码：', response.status_code)
    except Exception as e:
        print(e)
        time.sleep(60 + float(random.randint(1, 4000))/100)
        return get_json(headers,url)



def get_samples(html):
    samples = []
    user_id = re.findall('"profilePage_([0-9]+)"', html, re.S)[0]
    GIS_rhx_gis = re.findall('"rhx_gis":"([0-9a-z]+)"', html, re.S)[0]

    print('user_id：' + user_id)
    print(GIS_rhx_gis)
    doc = pq(html)
    items = doc('script[type="text/javascript"]').items()
    for item in items:
        if item.text().strip().startswith('window._sharedData'):
            # window._sharedData 的内容转换为字典
            js_data = json.loads(item.text()[21:-1], encoding='utf-8')
            
            # 12 张初始页面图片信息
            edges = js_data["entry_data"]["ProfilePage"][0]["graphql"]["user"]["edge_owner_to_timeline_media"]["edges"]
            # 网页页面信息
            page_info = js_data["entry_data"]["ProfilePage"][0]["graphql"]["user"]["edge_owner_to_timeline_media"]['page_info']
            # 下一页的索引值AQCSnXw1JsoV6LPOD2Of6qQUY7HWyXRc_CBSMWB6WvKlseC-7ibKho3Em0PEG7_EP8vwoXw5zwzsAv_mNMR8yX2uGFZ5j6YXdyoFfdbHc6942w
            cursor = page_info['end_cursor']
            # 是否有下一页
            flag = page_info['has_next_page']
            
            # 节点信息筛选
            for edge in edges:               
                
                # 如果是视频直接跳过
                if edge['node']['is_video'] == "true":
                    continue
                
                time.sleep(1)
                # 图片信息筛选
                sample = {}
                if edge['node']['display_url']:
                    display_url = edge['node']['display_url']
#                    print(display_url)
                    sample["img_url"] = display_url
                    sample["comment_count"] = edge['node']['edge_media_to_comment']["count"]
                    sample["like_count"] = edge['node']['edge_liked_by']["count"] 
                    print(sample["img_url"])
                    print(sample["comment_count"])
                    print(sample["like_count"])
                                                            
                if edge['node']['shortcode']:
                    shortcode = edge['node']['shortcode']
                    # https://www.instagram.com/p/{shortcode}/?__a=1
                    textUrl = 'https://www.instagram.com/p/' + shortcode + '/?__a=1'
                    textRespose = get_json(headers,textUrl)
#                    print(textRespose)
#                    print(type(textRespose))    
                    textDict = textRespose['graphql']['shortcode_media']['edge_media_to_caption']['edges'][0]['node']
                    sample["text"] = str(textDict)[10:-2]
                    print(sample["text"])
                    
                samples.append(sample)
                
            print(cursor, flag)
            
    # AJAX 请求更多信息                     
    while flag:
        url = uri.format(user_id=user_id, cursor=cursor)
        print(url)
        queryVariables = '{"id":"' + user_id + '","first":12,"after":"' +cursor+ '"}'
        print(queryVariables)
        headers['X-Instagram-GIS'] = hashStr(GIS_rhx_gis + ":" + queryVariables)
        print(headers)
        js_data = get_json(headers,url)
#        print(js_data)
        infos = js_data['data']['user']['edge_owner_to_timeline_media']['edges']
        cursor = js_data['data']['user']['edge_owner_to_timeline_media']['page_info']['end_cursor']
        flag = js_data['data']['user']['edge_owner_to_timeline_media']['page_info']['has_next_page']
        
#        print(infos)
        for info in infos:
            if info['node']['is_video']:
                continue
            else:
                sample = {}
                display_url = info['node']['display_url']
#                print(display_url)
                sample["img_url"] = display_url
                sample["comment_count"] = info['node']['edge_media_to_comment']["count"]
                sample["like_count"] = info['node']['edge_media_preview_like']["count"]                    
                                                        
                if info['node']['shortcode']:
                    time.sleep(1)
                    shortcode = info['node']['shortcode']
                    # https://www.instagram.com/p/{shortcode}/?__a=1
                    textUrl = 'https://www.instagram.com/p/' + shortcode + '/?__a=1'
                    textRespose = get_json(headers,textUrl)
#                    print(textRespose)
#                    print(type(textRespose))    
                    textDict = textRespose['graphql']['shortcode_media']['edge_media_to_caption']['edges'][0]['node']
                    sample["text"] = str(textDict)[10:-2]
                                        
                print(sample["img_url"])
                print(sample["comment_count"])
                print(sample["like_count"])  
                print(sample["text"])
                samples.append(sample)
                
        print(cursor, flag)
        
        # 下载120个 返回
        if len(samples) > 120:
            return samples

    return samples


def main():

    url = url_base
    html = get_html(url)
    samples = get_samples(html)
#    print(samples)
    with open("ins_samples.txt","a",encoding='utf-8') as f:
        f.write(str(samples))

In [4]:
#if __name__ == '__main__':
start = time.time()
main()

user_id：2158147977
00264b7a8b4328cd526648ca22138c4a
https://scontent-lht6-1.cdninstagram.com/vp/456614fb2e65223ba1f36d8ba7043d56/5C9CEFC1/t51.2885-15/e35/45748794_2169065799974414_7210616998919667712_n.jpg
111
10884
Boop 👆\n📷: @gettin_friggy_wit_it\n🐴: Horse\n#boopmynose\n•
https://scontent-lht6-1.cdninstagram.com/vp/0187a001460fe0f3ff52c061f221bf97/5CA0FCF4/t51.2885-15/e35/45751984_539371093139998_2388515924161331200_n.jpg
96
20706
Boop 👆\n📷: @larso1\n🐶: Corgi\n#boopmynose\n•
https://scontent-lht6-1.cdninstagram.com/vp/5c6b2f5bc16d02c76dadedad7b78803d/5CA574EF/t51.2885-15/e35/45920059_314474569278375_5087004989604233216_n.jpg
107
23074
Boop 👆\n📷: @morpheus_the_mixedeyed_dapple\n🐶: Dachshund\n#boopmynose\n•
https://scontent-lht6-1.cdninstagram.com/vp/050886aa4e008a95d8db2a156e00e3f4/5C92DCA8/t51.2885-15/e35/46287891_1978896609079029_5845733347812179968_n.jpg
117
23741
Boop 👆\n📷: @leongolden18\n🐶: Golden Retriever\n#boopmynose\n•
https://scontent-lht6-1.cdninstagram.com/vp/45e88862b722d

https://scontent-lht6-1.cdninstagram.com/vp/e83f07090b6a2e0392fad4e079aa4839/5C99935D/t51.2885-15/e35/45379926_523741234772614_5671545146622083072_n.jpg
561
37788
Boop 👆\n📷: @totalbullies\n🐶: English Bulldog\n#boopmynose\n•
https://scontent-lht6-1.cdninstagram.com/vp/3fd29c8fcc6c884d54c74b6ddc4483fe/5CABB9BE/t51.2885-15/e35/45344299_2245199995757828_3978432273618829312_n.jpg
157
24029
Boop 👆\n📷: @here_for_your_dog\n🐶: Labrador Retriever\n#boopmynose\n•
https://scontent-lht6-1.cdninstagram.com/vp/231def0e55e02400ddfe64c4ff067af7/5C9233B5/t51.2885-15/e35/43914262_377348706337414_3468613367751507968_n.jpg
100
22456
Boop 👆\n📷: @theshepherdcody\n🐶: Australian Shepherd\n#boopmynose\n•
https://scontent-lht6-1.cdninstagram.com/vp/53078646f7bb72cdf16b0fca9998c2a5/5CB012F0/t51.2885-15/e35/45378049_344481009713547_2047716380962193408_n.jpg
123
20773
Boop 👆\n📷: @thedoggolit\n🐶: Weimaraner\n#boopmynose\n•
https://scontent-lht6-1.cdninstagram.com/vp/606daacee966178afa1422881e85399b/5CACD81F/t51.2885

https://scontent-lht6-1.cdninstagram.com/vp/6396d6ddd386be48bef846ebb41b1418/5CAADA83/t51.2885-15/e35/44545019_546559092422724_5691821416643559424_n.jpg
119
22951
Boop 👆\n📷: @doug_the_nugg\n🐶: Corgi\n#boopmynose\n•
https://scontent-lht6-1.cdninstagram.com/vp/183437ca21994cfc6e911c58e5f39dac/5CA92C9F/t51.2885-15/e35/46155675_443161429550072_334751035245461504_n.jpg
158
21955
Boop 👆\n📷: @kelly_bove\n🐶: Border Collie\n#boopmynose\n•
https://scontent-lht6-1.cdninstagram.com/vp/81784d1fc084e228bc254d2481f159a7/5CA6CCFA/t51.2885-15/e35/45382446_259775184710525_8688734026812358656_n.jpg
91
21022
Boop 👆\n📷: @theothegreatdane\n🐶: Great Dane\n#boopmynose\n•
https://scontent-lht6-1.cdninstagram.com/vp/4b05f29d803e3e432175729a30414cab/5CAFA9D1/t51.2885-15/e35/44460183_655890271479454_6483219774301011968_n.jpg
193
25790
Boop 👆\n📷: @lilly_and_duncan\n🐶: Doberman Pinscher\n#boopmynose\n•
QVFDc2ZqY1VWaU9Uemx2S0J1RHR1ZVFha1YzbW44VFNhcTk1b2JwSnVLQ2o1RUQyVFFoRWk1SXA4ZU5XWklqZ1d4SkFNR084Mmx4TWFMLUtGS24xTn

https://scontent-lht6-1.cdninstagram.com/vp/643eafd84a0e531c3c15519462ffed76/5CAA7C7A/t51.2885-15/e35/44500661_447779129081509_3135612510204329984_n.jpg
88
19621
Boop 👆\n📷: @manukathemalamute\n🐶: Alaskan Malamute\n#boopmynose\n•
https://scontent-lht6-1.cdninstagram.com/vp/758a188a51706b8bbfe18b9943f19cfd/5CA5948B/t51.2885-15/e35/44515943_180390569566240_3919312439631413248_n.jpg
226
32072
Boop 👆\n📷: @gusthegolden8\n🐶: Golden Retriever\n#boopmynose\n•
https://scontent-lht6-1.cdninstagram.com/vp/05e2d1a816efc522f355ad1515ebc99f/5C93782D/t51.2885-15/e35/44511783_330576817726394_5529000228654415872_n.jpg
97
21224
Boop 👆\n📷: @coop.the.chocolate.lab\n🐶: Chocolate Labrador Retriever\n#boopmynose\n•
https://scontent-lht6-1.cdninstagram.com/vp/f51f7a63c73977f4bd18546fca368e67/5CAAD7E7/t51.2885-15/e35/44538824_2191931970819956_7274953629033824256_n.jpg
206
30000
Boop 👆\n📷: @bonnie_the_dobe\n🐶: Doberman Pinscher\n#boopmynose\n•
https://scontent-lht6-1.cdninstagram.com/vp/4dc1e40820d73441b31a11d73

https://scontent-lht6-1.cdninstagram.com/vp/049c7a21a42c87104544ac4c43ddc8ca/5CA0CEBB/t51.2885-15/e35/43603133_164126537875803_4033689527552835584_n.jpg
153
29081
Boop 👆\n📷: @young_charles_barkley\n🐶: Black Labrador Retriever\n#boopmynose\n•
https://scontent-lht6-1.cdninstagram.com/vp/787073c8e973c6be45f21bfe70b33236/5CA1568F/t51.2885-15/e35/42842886_2131683756871182_4495761118781767680_n.jpg
79
18147
Boop 👆\n📷: @guttendog.zeus\n🐶: Border Collie | GSD mix\n#boopmynose\n•
https://scontent-lht6-1.cdninstagram.com/vp/2df72ef8424c5fbe8735e1dd5c22d895/5CAFFB64/t51.2885-15/e35/42696476_533238527100940_2452845862716440576_n.jpg
177
27751
Boop 👆\n📷: @rach_bean\n🐶: Golden Retriever\n#boopmynose\n•
https://scontent-lht6-1.cdninstagram.com/vp/f8b234aa265ba0ba076708366edfddf1/5CAEC6CF/t51.2885-15/e35/44392027_1200847886731887_8772485966393769984_n.jpg
704
46174
Boop 👆\n📷: @kernelthegreatdane\n🐶: Great Dane\n#boopmynose\n•
https://scontent-lht6-1.cdninstagram.com/vp/6c77dae8c96825e205bc3c23da1c539f

In [7]:
print('🐷🐶🐱🐽🐮🐴')

🐷🐶🐱🐽🐮🐴


In [23]:
'🐷🐶🐱🐽🐮🐴'.encode('unicode-escape')

b'\\U0001f437\\U0001f436\\U0001f431\\U0001f43d\\U0001f42e\\U0001f434'

In [78]:
for i in range(10,99):
    emoj = "u'\x00\x01\xf4\x{}'".format(i)
    print(emoj)
    print(bytes(emoj.encode('raw_unicode_escape').decode('utf-8')))

SyntaxError: (unicode error) 'unicodeescape' codec can't decode bytes in position 14-15: truncated \xXX escape (<ipython-input-78-c3161810fda2>, line 2)

In [89]:
for i in range(0x1f3ff,0x1f450):
    print(chr(i),end=" ")

🏿 🐀 🐁 🐂 🐃 🐄 🐅 🐆 🐇 🐈 🐉 🐊 🐋 🐌 🐍 🐎 🐏 🐐 🐑 🐒 🐓 🐔 🐕 🐖 🐗 🐘 🐙 🐚 🐛 🐜 🐝 🐞 🐟 🐠 🐡 🐢 🐣 🐤 🐥 🐦 🐧 🐨 🐩 🐪 🐫 🐬 🐭 🐮 🐯 🐰 🐱 🐲 🐳 🐴 🐵 🐶 🐷 🐸 🐹 🐺 🐻 🐼 🐽 🐾 🐿 👀 👁 👂 👃 👄 👅 👆 👇 👈 👉 👊 👋 👌 👍 👎 👏 

In [55]:
u'\xe4\xbd\xa0\xe5\xa5\xbd'.encode('utf8')

b'\xc3\xa4\xc2\xbd\xc2\xa0\xc3\xa5\xc2\xa5\xc2\xbd'

In [56]:
u'\xe4\xbd\xa0\xe5\xa5\xbd'

'ä½\xa0å¥½'

In [67]:
b'\U0001f491'

b'\\U0001f491'

In [71]:
print( u'\U0001f491')

💑


In [ ]:
#https://emojipedia.org/dog-face/
#https://emojipedia.org/animoji/